In [1]:
import subprocess
import re
import pandas as pd

In [8]:
def run_GA(n_iterations: int, 
           n_individuals: int, 
           genome_length: int, 
           probability_crossover: float, 
           probability_mutation: float):
    executable = "./cmake-build-release/main"
    result = subprocess.run([executable, str(n_iterations), str(n_individuals), str(genome_length), str(probability_crossover), str(probability_mutation)], stdout=subprocess.PIPE)
    
    #print(f"Error code: {result.returncode}\n\n")
    
    results = pd.DataFrame(columns=['Best_idx','Best_fitness','Worst_idx','Worst_fitness','Avg_fitness','Best_genome'])
    regex_float = "[+-]?([0-9]*[.])?[0-9]+"
    regex_int = "[0-9]*"
    for line in result.stdout.decode("utf-8").split("\n"):
        match = re.search(f'Iter ({regex_int}): Best: ({regex_int})->({regex_float})  \|\| Worst: ({regex_int})->({regex_float})  \|\| Avg_fitness: ({regex_float})  \|\| Best_genome: ([\d ]*)', line)
        if match:
            results.loc[len(results.index)] = [match.group(2), match.group(3), match.group(5), match.group(6), match.group(8), match.group(10)]
    return results

In [27]:
n_iterations = 500
repetitions = 30
runs = {}
for individuals in [5, 10, 25]:
    for genome_length in [2,3,4]:
        for prob_crossover in range(0,12,2):
            for prob_mutation in range(0,12,2):
                id_run = f"{individuals} {genome_length} {prob_crossover/10} {prob_mutation/10}"
                runs[id_run] = {"i":individuals, "gl":genome_length, "pc":prob_crossover/10, "pm":prob_mutation/10}
#print(runs)
#result = run_GA(100, 2, 2, 1, 1)

In [28]:
from multiprocessing import Pool
import tqdm


def f(x):
    row = runs[x]
    result = []
    for i in range(repetitions):
        result.append(run_GA(n_iterations, row["i"], row["gl"], row["pc"], row["pm"]))
    return x, result
with Pool(12) as p:
    results_run = list(tqdm.tqdm(p.imap(f, runs), total=len(runs)))

100%|█████████████████████████████████████████| 324/324 [12:59<00:00,  2.40s/it]


In [29]:
print(len(results_run))

324


In [30]:
import pickle
with open('results_5_10_25_individuals.pickle', 'wb') as handle:
    pickle.dump(results_run, handle, protocol=pickle.HIGHEST_PROTOCOL)

# Load data (deserialize)
#with open('filename.pickle', 'rb') as handle:
#    unserialized_data = pickle.load(handle)

In [25]:
def get_best_result_run(run):
    best_idx = 1000
    ending = 100000000
    for idx, it in enumerate(run[1]):
        if float(it.iloc[-1,:]["Best_fitness"]) < ending:
            ending = float(it.iloc[-1,:]["Best_fitness"])
            genome = it.iloc[-1,:]["Best_genome"]
            best_idx = idx
    return ending, genome
        
    

In [ ]:
for run in results_run:
    print(run[0], get_best_result_run(run))d